In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
iris = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
iris.species

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object

In [4]:
mappings = {
   'setosa': 0,
   'versicolor': 1,
   'virginica': 2
}
iris['species'] = iris['species'].apply(lambda x: mappings[x])

In [5]:
X = iris.drop('species', axis=1).values
y = iris['species'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [36]:
class ANN(nn.Module):
    def __init__(self):
       super().__init__()
       self.fc1 = nn.Linear(in_features=4, out_features=16)
       self.fc2 = nn.Linear(in_features=16, out_features=12)
       self.output = nn.Linear(in_features=12, out_features=3)
 
    def forward(self, x):
        x=self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = F.relu(self.fc2(x))
        x = self.output(x)
        
        return x

In [37]:
model = ANN()
model

ANN(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=12, bias=True)
  (output): Linear(in_features=12, out_features=3, bias=True)
)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [45]:
%%time
epochs = 200
loss_arr = []
for i in range(epochs):
    y_hat = model(X_train)
    loss = criterion(y_hat, y_train)
    loss_arr.append(loss)
    
    if i % 10 == 0:
        print(f'Epoch: {i} Loss: {loss}')
 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0 Loss: 0.03302381932735443
Epoch: 10 Loss: 0.03659069910645485
Epoch: 20 Loss: 0.02831370010972023
Epoch: 30 Loss: 0.04809735715389252
Epoch: 40 Loss: 0.03626380115747452
Epoch: 50 Loss: 0.043814897537231445
Epoch: 60 Loss: 0.036404483020305634
Epoch: 70 Loss: 0.03696012869477272
Epoch: 80 Loss: 0.03378427401185036
Epoch: 90 Loss: 0.05111926048994064
Epoch: 100 Loss: 0.03467506542801857
Epoch: 110 Loss: 0.030420901253819466
Epoch: 120 Loss: 0.028775719925761223
Epoch: 130 Loss: 0.04030948877334595
Epoch: 140 Loss: 0.03260606527328491
Epoch: 150 Loss: 0.032477330416440964
Epoch: 160 Loss: 0.036996666342020035
Epoch: 170 Loss: 0.037058163434267044
Epoch: 180 Loss: 0.0379277803003788
Epoch: 190 Loss: 0.03862924873828888
CPU times: total: 1.59 s
Wall time: 396 ms


In [46]:
preds = []
with torch.no_grad():
    for val in X_test:
        y_hat = model(val)
        preds.append(y_hat.argmax().item())

In [47]:
preds

[1,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0]

In [48]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]

In [49]:
df['Correct'].sum() / len(df)

1.0